In [45]:
# needed for GridSearch
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, KFold, GridSearchCV
from sklearn import metrics
from pathlib import Path
from sklearn.linear_model import LogisticRegressionCV
from sklearn.linear_model import LogisticRegression

path = Path('data', 'weatherAUS_clean.csv')
df = pd.read_csv(path)

In [46]:
# split into train and test
label = df.pop('RainTomorrow_Yes')
train_X, test_X, train_y, test_y = train_test_split(df, label, test_size= 0.3)

In [50]:
# Logistic Regression

# different values for most important hyper-parameters
parameter_space = {
    'solver': ['newton-cg', 'lbfgs', 'liblinear'],
    'penalty': ['none', '11', '12', 'elasticnet'],
    'C': [0.001, 1, 1000],
}

clf = LogisticRegression()
clf = GridSearchCV(clf, parameter_space, n_jobs=-1, cv=5)
clf.fit(train_X, train_y)

print('Best parameters found: ', clf.best_params_)
print('Mean-CV accuracy: %0.3f' % clf.best_score_)

predicitons = clf.predict(test_X)
print(metrics.classification_report(test_y, predicitons, digits=3))


C:\Users\ronal\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
60 fits failed out of a total of 72.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
6 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\ronal\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\ronal\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 1091, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "C:\Users\ronal\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 78, in _check_solver
    raise ValueError("penalt

Best parameters found:  {'C': 0.001, 'penalty': 'none', 'solver': 'newton-cg'}
Mean-CV accuracy: 0.853
              precision    recall  f1-score   support

           0      0.871     0.945     0.906     18177
           1      0.745     0.536     0.623      5472

    accuracy                          0.850     23649
   macro avg      0.808     0.740     0.765     23649
weighted avg      0.842     0.850     0.841     23649



dtype('int64')